In [1]:
import numpy as np
import subprocess as sp
from openeye.oechem import *
from openforcefield.typing.engines.smirnoff import *
from networkx.algorithms import isomorphism

# Load molecules into a list of `OEMol`s

I'm going to start by loading the host and guest SYBYL-formatted `mol2` files into a list of `OEMol`s. I created these SYBYL-formatted files from files containing AM1-BCC charges and GAFF v1.7 Lennard-Jones and bonded parameters:

```
antechamber -i bcd.mol2 -fi mol2 -o bcd-sybyl.mol2 -fo mol2 -at sybyl
antechamber -i ben.mol2 -fi mol2 -o ben-sybyl.mol2 -fo mol2 -at sybyl -dr n 
# Disable `acdoctor` to handle carboxylate group
```

I think it should be possible to read in GAFF-formatted files directly, using a specific forcefield flavor, but I don't see the relvant flavor listed in their [documentation](https://docs.eyesopen.com/toolkits/python/oechemtk/molreadwrite.html#section-molreadwrite-flavoredinputandoutput). A search for "GAFF" comes up empty (except for one blog post). If I try to load the GAFF files without any special flavor, I get many things with atom type "Du," which I surmise to be a dummy atom type. I believe this can be remedied by running the files through `OETriposAtomNames`, but it is another thing that could go wrong, so for these reasons, I believe it is more straight-forward to simply start with standard atom names.

In [24]:
def load_mol2(filename, name=None, add_tripos=True):
    ifs = oemolistream()
    if not ifs.open(filename):
        print(f'Unable to open {filename} for reading...')
    for mol in ifs.GetOEMols():
        if add_tripos:
            OETriposAtomNames(mol)
        if name:
            mol.SetTitle(name)
    return mol

In [25]:
host = load_mol2('original/bcd-sybyl.mol2', name='bcd', add_tripos=False)

In [26]:
guest = load_mol2('original/ben-sybyl.mol2', name='ben', add_tripos=False)

In [27]:
molecules = [host, guest]

Now, I'm going to add water and ions to the molecules list.

In [28]:
def add_waters_and_ions():
    smiles = ['[Na+]', '[Cl-]', 'O']
    molecules = []
    for molecule in smiles:
        molecules.append(process_smiles(molecule))
    return molecules

def process_smiles(string, name=None, add_hydrogens=True, add_tripos=True):
    mol = OEMol()
    OESmilesToMol(mol, string)
    if add_hydrogens:
        OEAddExplicitHydrogens(mol)
    if add_tripos:
        OETriposAtomNames(mol)
    if name:
        mol.SetTitle(name)
    return mol

In [29]:
water_and_ions = add_waters_and_ions()

In [30]:
molecules += water_and_ions

# Add charges (?)

In [33]:
tmp = molecules[0]

In [34]:
tmp.GetAtom(OEHasAtomIdx(1)).GetPartialCharge()

AttributeError: 'NoneType' object has no attribute 'GetPartialCharge'

In [38]:
tmp.GetMaxAtomIdx()

0

In [40]:
tmp.GetAtom(OEHasAtomIdx(10))